In [35]:
import pandas as pd
from pyecharts import options as opts
from pyecharts.charts import Pie,Bar,Map,Line
from collections import Counter
import numpy as np
path = '/Users/bruce/Desktop/build_database/产品.xls'

In [68]:
# 学历要求硕士及以上 和 学历要求本科及以上的平均薪资
data = pd.read_excel(path)
data = data[['学历要求','平均薪资/k']]
data = data.groupby('学历要求').mean()
data

,平均薪资/k
学历要求,
大专及以上,13.948675
学历不限,17.697500
本科及以上,21.828140
硕士及以上,21.833333


In [75]:
# 职位诱惑'弹性' 占比多少？
data = pd.read_excel(path)
data_list = [] 
for i in data['职位诱惑统计词']:
    data_list = data_list + eval(i)
print('弹性占比 {}%'.format(dict(Counter(data_list))['弹性'] / len(data_list)*100) )

弹性占比 1.8150837161060898%


In [131]:
# 新建一列最高薪和最低薪的差值
data = pd.read_excel(path)
data['差值'] = np.subtract(data['最高薪资/k'],data['最低薪资/k'])
data['差值']

0       15
1       15
2        8
3       10
4       20
5        5
6        7
7       10
8       15
9       10
10      15
11       7
12      10
13       5
14      10
15       1
16       5
17       4
18       4
19      10
20       4
21       6
22       5
23      10
24      10
25      15
26       7
27      20
28       2
29      15
        ..
2623     5
2624    20
2625    20
2626    20
2627    30
2628    15
2629    10
2630    20
2631     8
2632    10
2633    10
2634    20
2635    10
2636    20
2637    20
2638    30
2639     4
2640    35
2641    12
2642    20
2643    10
2644    35
2645    10
2646    20
2647    20
2648    30
2649    10
2650    25
2651    15
2652    20
Name: 差值, Length: 2653, dtype: int64

In [239]:
# 看处于不同发展阶段 对产品经理的需求量，以及对学历的要求程度，薪资水平
data = pd.read_excel(path)
data1 = data['发展阶段'].value_counts() 
xaxis = list(data1.keys()[:8]) 
temp_dict = {}
for i in xaxis:
    temp_dict[i] ='' 


data2 = data[['发展阶段','平均薪资/k']]
data2 = data2.groupby('发展阶段').mean()
for i in temp_dict.keys():
    temp_dict[i] = data2['平均薪资/k'][i]
data2_list = temp_dict.values() # 薪资关系


data3 = data[['发展阶段','学历要求']]
data3['辅助列'] = [1 for i in range(len(data['发展阶段']))]
data3 = data3.groupby(['发展阶段','学历要求']).count()
dazhuan = []
buxian = []
benke = []
shuoshi = []
def add(list1,i,require):
    try:
        list1.append(data3['辅助列'][i][require])
    except:
        list1.append(0)
for i in temp_dict.keys():
    add(dazhuan,i,'大专及以上 ')
    add(buxian,i,'学历不限 ')
    add(benke,i,'本科及以上 ')
    add(shuoshi,i,'硕士及以上 ')

c = (
        Line()
        .add_xaxis(xaxis)
        .add_yaxis("对产品的需求量", list(map(int,data1.values[0:8])))  # 不接受np的int64
        .add_yaxis("薪资", data2_list)
        .add_yaxis("大专及以上", list(map(int,dazhuan)))
        .add_yaxis("学历不限", list(map(int,buxian)))
        .add_yaxis("本科及以上", list(map(int,benke)))
        .add_yaxis("硕士及以上", list(map(int,shuoshi)))
        .set_global_opts(title_opts=opts.TitleOpts(title="公司不同发展阶段的要求",pos_top = "5%"))
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))  # 不显示标签,即图上不显示数字,不然很乱
    )
c.render_notebook()

/Users/bruce/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [172]:
# 不同领域的薪资情况
data = pd.read_excel(path)
data = data[['领域','平均薪资/k']]
data = data.groupby('领域').mean()

data = sorted(dict(data['平均薪资/k']).items(),key=lambda item:item[1],reverse=True)[:10]

c = (
    Pie()
    .add('领域和薪资关系',data)
    .set_global_opts(title_opts=opts.TitleOpts(title="领域和薪资关系",pos_top = "20%"))
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
    
).render_notebook()
c

In [37]:
# 判断起薪高的城市其最高薪资是否也高
data = pd.read_excel(path)
data = data[['工作地点','最低薪资/k','最高薪资/k']]
data = data.groupby('工作地点').mean()
data1 = sorted(dict(data['最低薪资/k']).items(),key=lambda item:item[1],reverse=True)[:10]
xaxis = []
min_money_list = []
max_money_list = []
for i in data1:
    xaxis.append(i[0])
    min_money_list.append(i[1])
    max_money_list.append(data['最高薪资/k'][i[0]])

c = (
        Line()
        .add_xaxis(xaxis)
        .add_yaxis("起薪", min_money_list)
        .add_yaxis("最高薪资", max_money_list)
        .set_global_opts(title_opts=opts.TitleOpts(title="不同城市 起薪与最高薪资的关系"))
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))  # 不显示标签,即图上不显示数字,不然很乱
    )
c.render_notebook()

In [192]:
# 经验要求与公司规模之间的关系
data = pd.read_excel(path)
data = data[['公司规模','经验要求']]
data['辅助列'] = [1 for i in range(len(data['公司规模']))] 

data1 = data['公司规模'].value_counts()
xaxis = list(data1.keys()) 
temp_dict = {}
for i in xaxis[:-1]:
    temp_dict[i] ='' 


data = data.groupby(['公司规模','经验要求']).count()


one_three= []
more_ten = []
less_one = []
three_five = []
five_teen = []
buxian = []
yingjie = []
def add(list1,i,require):
    try:
        list1.append(data['辅助列'][i][require])
    except:
        list1.append(0)
for i in temp_dict.keys():
    add(one_three,i,'经验1-3年 ')
    add(more_ten,i,'经验10年以上 ')
    add(less_one,i,'经验1年以下 ')
    add(three_five,i,'经验3-5年 ')
    add(five_teen,i,'经验5-10年 ')
    add(buxian,i,'经验不限 ')
    add(yingjie,i,'经验应届毕业生 ')
    
c = (
        Line()
        .add_xaxis(xaxis[:-1])
        .add_yaxis("经验1-3年", list(map(int,one_three)))  # 不接受np的int64
        .add_yaxis("经验10年以上", list(map(int,more_ten)))
        .add_yaxis("经验1年以下", list(map(int,less_one)))
        .add_yaxis("经验3-5年", list(map(int,three_five)))
        .add_yaxis("经验5-10年", list(map(int,five_teen)))
        .add_yaxis("经验不限", list(map(int,buxian)))
        .add_yaxis("经验应届毕业生", list(map(int,yingjie)))
        .set_global_opts(title_opts=opts.TitleOpts(title="公司不同发展阶段的要求",pos_top = "5%"))
        .set_series_opts(label_opts=opts.LabelOpts(is_show=False))  # 不显示标签,即图上不显示数字,不然很乱
    )
c.render_notebook()

In [193]:
# 有产品经理需求的城市  前6名
data = pd.read_excel(path)
work_adress = dict(Counter(data['工作地点']))
work_adress = sorted(work_adress.items(),key=lambda item:item[1],reverse=True)[:6]

c = (
    Pie()
    .add('产品经理职位分布',work_adress)
    .set_global_opts(title_opts=opts.TitleOpts(title="产品职位排名前6的城市"))
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
    
).render_notebook()
c

In [269]:
# 懂数据的产品薪资是否偏高
data = pd.read_excel(path)
data = data[['职位名称','平均薪资/k']]
index_list = []
for i,j in enumerate(data['职位名称']):
    if '数据产品' in j:
        index_list.append(i)
print('数据产品平均薪资:{}'.format(data.iloc[index_list].mean()))

other_list = []
for i in range(len(data['职位名称'])):
    if i not in index_list:
        other_list.append(i)

print('其它产品平均薪资:{}'.format(data.iloc[other_list].mean()))

数据产品平均薪资:平均薪资/k    26.15331
dtype: float64
其它产品平均薪资:平均薪资/k    19.948648
dtype: float64


In [197]:
# 游戏领域的产品岗占比大致多少
data = pd.read_excel(path)
print("游戏领域 产品岗占比{}%".format(data['领域'].value_counts()['游戏'] / len(data['领域']) *100))

游戏领域 产品岗占比4.749340369393139%


In [200]:
# 同一个城市不同地区的工资差别
data = pd.read_excel(path)
data = data[['工作地点','工作地址 1','平均薪资/k']]
data.groupby(['工作地点','工作地址 1']).mean()

平均薪资/k
工作地点 工作地址 1              
上海   嘉定区        21.916667
     宝山区        18.833333
     徐汇区        18.889706
     普陀区        16.666667
     杨浦区        22.261905
     松江区        15.166667
     浦东新区       23.995652
     虹口区        17.850000
     长宁区        24.362500
     闵行区        23.534091
     闸北区        15.000000
     青浦区        19.611111
     静安区        17.920000
     黄浦区        23.020833
东莞   东莞市市辖区      9.500000
中山   中山市市辖区      5.000000
佛山   南海区        14.000000
     禅城区         9.600000
     顺德区        21.250000
北京   东城区        33.068182
     丰台区        24.714286
     大兴区        26.045455
     昌平区        19.500000
     朝阳区        25.152819
     海淀区        25.359420
     石景山区       32.800000
     西城区        22.714286
     通州区        27.250000
南京   建邺区        28.333333
     栖霞区        29.000000
...                   ...
福州   台江区        20.000000
     鼓楼区        15.000000
绵阳   游仙区        10.500000
苏州   吴中区        12.583333
     工业园区       30.000000
     昆山市        30.000000
     虎丘区        12.375000
     高新区        15.000000
西安   雁塔区        11.900000
     高新技术产业开发区  10.600000
郑州   中原区        21.250000
     二七区        10.000000
     郑东新区       10.375000
     金水区        11.000000
     高新区         6.000000
重庆   南岸区        11.500000
     巴南区         6.500000
     涪陵区         4.500000
     渝中区         5.000000
     渝北区        11.611111
     荣昌区        12.500000
长沙   天心区        12.500000
     岳麓区        13.714286
     开福区        12.500000
     望城区        11.500000
     长沙县        16.500000
     雨花区        12.400000
青岛   市南区        14.250000
黄石   西塞山区       30.000000
黔南   独山县         3.500000

[144 rows x 1 columns]

In [202]:
#  规模>=500 公司一般都是什么薪资水平？
data = pd.read_excel(path)
data = data[['公司规模','平均薪资/k']]
data.groupby('公司规模').mean()

,平均薪资/k
公司规模,
15-50人,14.386740
150-500人,19.870098
2000人以上,26.199021
50-150人,17.881786
500-2000人,23.139073
少于15人,13.743902
紫金科创(天使轮)，江苏舜天(天使轮)，金陵饭店(天使轮)，江苏高科技投资集团(天使轮),20.000000


In [210]:
# 所有产品经理岗位对经验要求的比例分布情况。
data = pd.read_excel(path)
data = data['经验要求'].value_counts()
data_list = []
count = sum(data.values)
for i,j in data.items():
    data_list.append((i,round(j/count*100,2)))
c = (
    Pie()
    .add('经验要求占比 %',data_list)
    .set_global_opts(title_opts=opts.TitleOpts(title="经验要求占比 %",pos_top = "5%"))
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
    
).render_notebook()
c

In [212]:
# 产品经理岗位所涉猎的前10个领域有哪些。 
data = pd.read_excel(path)
data = data['领域'].value_counts()
sorted(data.items(),key=lambda item:item[1],reverse=True)[:10]


[('移动互联网', 276),
 ('电商', 145),
 ('移动互联网,电商', 139),
 ('游戏', 126),
 ('金融', 102),
 ('移动互联网,游戏', 86),
 ('移动互联网,教育', 77),
 ('移动互联网,企业服务', 75),
 ('文娱丨内容', 74),
 ('教育', 73)]

In [214]:
# 什么发展阶段的公司对产品经理的需求最大？
data = pd.read_excel(path)
data = data['发展阶段'].value_counts()
sorted(data.items(),key=lambda item:item[1],reverse=True)[:1]

[('不需要融资', 749)]

In [215]:
# 什么城市对产品经理的需求最大？
data = pd.read_excel(path)
work_adress = dict(Counter(data['工作地点']))
sorted(work_adress.items(),key=lambda item:item[1],reverse=True)[:1]

[('北京', 799)]

In [232]:
# 产品经理的岗位，全职的薪水top5的公司
data = pd.read_excel(path)
data = data[['公司名称','工作性质','平均薪资/k']].loc[data['工作性质']=='全职']
data = data.groupby('公司名称').mean()
sorted(data['平均薪资/k'].items(),key=lambda item:item[1],reverse=True)[:5]

[('仁线科技', 115.0),
 ('慧泊金科技', 90.0),
 ('酷划在线', 80.0),
 ('和智未来', 75.0),
 ('广州虎牙科技有限公司', 75.0)]

In [234]:
# 产品经理D轮及以上公司需要的技能要求
data = pd.read_excel(path)
data = data[['技能要求']]['技能要求'].loc[data['发展阶段']=='D轮及以上'] # 这里一次索引 得到的是dataframe 需要二次索引得到列  当然也可以直接data['技能要求']   我这里只是做个记录 有这么个用法
data_list = []
for i in data:
    data_list = data_list + i.split(sep = '，')

work_skill = dict(Counter(data_list)) 
# 数据这里  本科及以上学历 和 本科以上学历需要合并一下
work_skill['本科及以上学历'] = work_skill['本科及以上学历'] + work_skill['本科以上学历']
work_skill.pop('本科以上学历')
work_skill = sorted(work_skill.items(),key=lambda item:item[1],reverse=True)[:10]

c = (
    Pie()
    .add('产品核心技能要求',work_skill)
    .set_global_opts(title_opts=opts.TitleOpts(title="D轮及以上公司对产品的要求",pos_top = "20%"))
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
    
).render_notebook()
c

In [238]:
# 北京地区在招产品经理，经验在1-3年的职位要求
data = pd.read_excel(path)
data = data[['工作地点','经验要求','职位要求']].loc[data['工作地点']=='北京']
data = data[data['经验要求']=='经验1-3年 ']
data

,工作地点,经验要求,职位要求
29,北京,经验1-3年,职位描述: 工作职责1、 负责AI内容风控产品，包括产品线搭建、需求分析、与功能迭代等2、通...
31,北京,经验1-3年,职位描述: 岗位职责：1、负责薪人薪事SaaS产品的规划设计，把握整体前进方向；2、善于研究...
38,北京,经验1-3年,职位描述: 工作职责：1、负责基于金融风控业务相关数据产品的数据需求分析、原型设计、业务上线...
57,北京,经验1-3年,职位描述: 工作职责：1、调研公司竞品，了解产品需求，撰写产品技术方案2、参与公司招投标及商...
65,北京,经验1-3年,职位描述: 岗位职责：1. 负责金融业务相关数据类和系统类产品的需求分析和产品策划工作；2...
71,北京,经验1-3年,职位描述: 1、 负责AI内容风控产品，包括产品线搭建、需求分析、与功能迭代等2、通过行业洞...
112,北京,经验1-3年,职位描述: 职位描述：1) 负责医学大数据平台及应用产品的规划和设计；2) 与客户及生物医疗...
140,北京,经验1-3年,职位描述: 职位职责： 1、负责公司情侣产品的功能设计、优化，对用户画像和用户需求进行深入研...
147,北京,经验1-3年,职位描述: 工作职责1.根据产品功能设计产品需求文档及产品策略，并评估产品迭代的效果 ；2....
153,北京,经验1-3年,职位描述: 职位描述： 1、完成需求分析、功能策划，关注竞品，定义需求优先级； 2、分析业...
